# O Projeto

- Criar as consultas utilizando linguagem T-SQL
- Entregar o código gerado no teste com o passo a passo no README.md

Com base no modelo de dados, será feita uma série de análises que visam retornar os dados corretos conforme a base criada segundo o diagrama abaixo:

![<img src="samples/model.png" height="500" alt="Modelo" title="Modelo"/>](samples/model.png)

In [6]:
# Import Modules

import sqlite3
import pandas as pd

# Criando a conexão com a base

conn = sqlite3.connect('bike_stores.db')
cursor = conn.cursor()

In [9]:
# listanddo os clientes que não fizeram nenhuma compra

# Listar todos Clientes que não tenham realizado uma compra
cursor.execute('''
    SELECT * FROM customers
    WHERE customer_id NOT IN (SELECT DISTINCT customer_id FROM orders);
''')

no_purchase_customers = cursor.fetchall()
no_purchase_customers_df = pd.DataFrame(no_purchase_customers, columns=['customer_id', 'first_name', 'last_name', 'phone', 'email', 'street', 'city', 'state', 'zip_code'])

no_purchase_customers_df


,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,6,Jessica,Pearson,4444444444,jessica@example.com,1313 Cedar St,CityF,StateC,67890
1,7,Louis,Litt,5555555555,louis@example.com,1515 Maple St,CityG,StateD,78901
2,8,Donna,Paulsen,6666666666,donna@example.com,1717 Birch St,CityH,StateE,89012
3,9,Sheila,Sazs,7777777777,sheila@example.com,1919 Walnut St,CityI,StateF,90123
4,10,Jeff,Malone,8888888888,jeff@example.com,2121 Cherry St,CityJ,StateG,12321


In [10]:
# Listagem dos produtos que não foram vendidos
cursor.execute('''
    SELECT * FROM products
    WHERE product_id NOT IN (SELECT DISTINCT product_id FROM order_items);
''')
no_purchase_products = cursor.fetchall()
no_purchase_products_df = pd.DataFrame(no_purchase_products, columns=['product_id', 'product_name', 'brand_id', 'category_id', 'model_year', 'list_price'])

no_purchase_products_df

,product_id,product_name,brand_id,category_id,model_year,list_price
0,6,Product F,6,6,2023,599.99
1,7,Product G,7,7,2022,799.99
2,8,Product H,8,8,2021,899.99
3,9,Product I,9,9,2020,1099.99
4,10,Product J,10,10,2023,1299.99


In [10]:
# Listagem dos produtos sem itens em estoque
cursor.execute('''
    SELECT p.*, COALESCE(s.quantity, 0) AS quantity
    FROM products p
    LEFT JOIN stocks s ON p.product_id = s.product_id
    WHERE COALESCE(s.quantity, 0) = 0;
''')
no_stock_products = cursor.fetchall()
no_stock_products_df = pd.DataFrame(no_stock_products, columns=['product_id', 'product_name', 'brand_id', 'category_id', 'model_year', 'list_price', 'quantity'])

no_stock_products_df

,product_id,product_name,brand_id,category_id,model_year,list_price,quantity
0,3,Product C,3,3,2023,299.99,0
1,4,Product D,4,4,2021,199.99,0
2,6,Product F,6,6,2023,599.99,0
3,7,Product G,7,7,2022,799.99,0


In [12]:
# Agrupar a quantidade de vendas que cada marca realizou

cursor.execute('''
    SELECT b.brand_name, s.store_name, COUNT(oi.product_id) AS total_sales
    FROM order_items oi
    JOIN products p ON oi.product_id = p.product_id
    JOIN brands b ON p.brand_id = b.brand_id
    JOIN orders o ON oi.order_id = o.order_id
    JOIN stores s ON o.store_id = s.store_id
    GROUP BY b.brand_name, s.store_name;
''')
brand_sales_per_store = cursor.fetchall()
brand_sales_per_store_df = pd.DataFrame(brand_sales_per_store, columns=['brand_name', 'store_name', 'total_sales'])


brand_sales_per_store_df

,brand_name,store_name,total_sales
0,Brand A,Store A,1
1,Brand B,Store B,1
2,Brand C,Store C,1


In [15]:
# Listar os funcionários que não estão relacionados a nenhum pedido

cursor.execute('''
    SELECT * FROM staffs
    WHERE staff_id NOT IN (SELECT DISTINCT staff_id FROM orders);
''')
no_order_staffs = cursor.fetchall()
no_order_staffs_df = pd.DataFrame(no_order_staffs, columns=['staff_id', 'first_name', 'last_name', 'email', 'phone', 'active', 'store_id', 'manager_id'])

no_order_staffs_df

,staff_id,first_name,last_name,email,phone,active,store_id,manager_id
0,4,Anna,Davis,anna.davis@example.com,6666666666,1,4,2
1,5,Robert,Wilson,robert.wilson@example.com,5555555555,1,5,2
